In [1]:
################################
# install
################################
! pip install pandas

In [16]:
################################
# imports
################################
import pandas as pd
import numpy as np
import os
import re
import nltk
from deep_translator import GoogleTranslator
from langdetect import detect
from tqdm import tqdm  #
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
# Download necessary NLTK datasets
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vdinh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vdinh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vdinh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
################################
# test variable
################################

test = True

In [ ]:
################################
# load in the world new data
################################

current_dir = os.getcwd()

if test:
    print('current_dir: ' + str(current_dir) + '\n')

#csv_path = current_dir + "\Top_25_World_News_2018_2023\WorldNewsData.csv"
csv_path = r"{}\Top_25_World_News_2018_2023\WorldNewsDataV2.csv".format(current_dir)

if test:
    print('csv_path: ' + str(csv_path) + '\n')

# convert csv to pandas dataframe
news_data = pd.read_csv(csv_path)

if test:
    print('len(news_data)    :' + str(len(news_data))    + '\n')
    print('news_data.iloc[0] :' + str(news_data.iloc[0]) + '\n')
    print('news_data.head()  :' + str(news_data.head())  + '\n')

In [ ]:
################################
# load in the stock data
################################

current_dir = os.getcwd()
dirs = [
        os.path.join(os.getcwd(), "Stock_Market_Data_NASDAQ_NYS_S&P500", "stock_market_data", "forbes2000"),
        os.path.join(os.getcwd(), "Stock_Market_Data_NASDAQ_NYS_S&P500", "stock_market_data", "nasdaq"),
        os.path.join(os.getcwd(), "Stock_Market_Data_NASDAQ_NYS_S&P500", "stock_market_data", "nyse"),
        os.path.join(os.getcwd(), "Stock_Market_Data_NASDAQ_NYS_S&P500", "stock_market_data", "sp500"),
       ]

#csv_files = [os.path.join(csv_dir, file) for file in os.listdir(csv_dir)]

if test:
    print('current_dir: ' + str(current_dir) + '\n')
    #print('csv_files  : ' + str(csv_files) + '\n')

# save list to handle duplicate stocks
processed_files = set()
stock_data      = []

# convert csv to pandas dataframe then combine
for dir in dirs:
    csv = [os.path.join(dir, file) for file in os.listdir(dir)]
    for file in csv:
        # take only first part of file name (leave out csv)
        file_name = os.path.splitext(os.path.basename(file))[0]
        # if duplicate, continue and skip
        if file_name in processed_files:
            continue
        # read in file and append
        df = pd.read_csv(file)
        df['Stocks'] = file_name
        stock_data.append(df)
        processed_files.add(file_name)

stock_data = pd.concat(stock_data, ignore_index=True)

if test:
    print('len(stock_data)    :' + str(len(stock_data))    + '\n')
    print('stock_data.iloc[0] :' + str(stock_data.iloc[0]) + '\n')
    print('stock_data.iloc[1] :' + str(stock_data.iloc[1]) + '\n')
    print('stock_data.head()  :' + str(stock_data.head()) + '\n')

In [ ]:
#######################################################
# remove dates older than May 01, 2018 for stock data
#######################################################

# stock_data.iloc[1] :Date              19-11-1999
stock_data['Date'] = pd.to_datetime(stock_data['Date'], format='%d-%m-%Y')
filt_new_data = stock_data[stock_data['Date'] >= '2018-05-01']
filt_new_data = filt_new_data.reset_index(drop=True)

if test:
    print('len(filt_new_data)    :' + str(len(filt_new_data))    + '\n')
    print('filt_new_data.head()  :' + str(filt_new_data.head())  + '\n')

In [ ]:
# convert news data format to stocks data

#news_data.iloc[0] :Date                                          May 01, 2018
news_data2 = news_data.copy()
#news_data2['Date'] = pd.to_datetime(news_data2['Date'], infer_datetime_format=True, errors='coerce')
#news_data2['Date Date'] = pd.to_datetime(news_data2['Date']).dt.strftime('%d-%m-%y')
news_data2['Date'] = pd.to_datetime(news_data2['Date'], format='%b %d, %Y', errors='coerce').dt.strftime('%d-%m-%y')

if test:
    print('len(news_data2)      :' + str(len(news_data2))      + '\n')
    print('news_data2.iloc[50] :' + str(news_data2.iloc[50])   + '\n')
    print('news_data2.head()    :' + str(news_data2.head())    + '\n')


In [ ]:
news_data2.to_csv('updated_news_data.csv')
filt_new_data.to_csv('updated_stock_data.csv')

In [ ]:
pip install deep-translator langdetect

In [ ]:
# Load the CSV file
file_path = "updated_news_data.csv" 
df = pd.read_csv(file_path)

columns_to_translate = df.columns[2:27]

def translate_text(text):
    try:
        if pd.notna(text):  # Check if the text is not NaN
            lang = detect(text)
            if lang != "en":  # Translate only if not English
                return GoogleTranslator(source=lang, target="en").translate(text)
        return text  
    except:
        return text  
for col in tqdm(columns_to_translate, desc="Translating Columns"):
    df[col] = df[col].astype(str).apply(translate_text)

translated_file_path = "translated_news_data.csv"
df.to_csv(translated_file_path, index=False)

print(f"Translation complete. Saved to {translated_file_path}")


In [ ]:
# Next part, human eyes are needed in order to translate what is missed in the previous step.
# The new .csv file is called human_translated_news_data.csv

In [18]:
df = pd.read_csv('human_translated_news_data.csv')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def normalize_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
        tokens = word_tokenize(text)
        normalized_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
        return ' '.join(normalized_tokens)
    else:
        # Return empty string for non-string data
        return ''

# Normalize each headline in Top1 to Top25 columns
for col in df.columns[2:]:  # Skip the first two columns (Unnamed and Date)
    df[col] = df[col].apply(normalize_text)

df.to_csv('normalized_headlines.csv', index=False)
print(df.head())  

   Unnamed: 0      Date                                               Top1  \
0           0  1-May-18    north korea open sky south korean medium report   
1           1  2-May-18  got fear trump threatens declassify fbi justic...   
2           2  3-May-18  hollywood isnt simply churning crummy remake n...   
3           3  4-May-18  donald trump say london hospital like war zone...   
4           4  5-May-18  missing teen mom implores canada take note u s...   

                                                Top2  \
0             mueller probe aint ending anytime soon   
1  best garage heater review incuding electric ga...   
2  iran foreign minister u consistently violated ...   
3  german seat un security council likely israel ...   
4      william kate release new picture prince louis   

                                                Top3  \
0  brand new 2018renault alpine a110startup amp l...   
1  apple fueled everyones biggest fear company go...   
2  rihanna cover june 2018